# Day 3: 3.2

## XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

## Here we are creating an DeepLearning model that can predict wether the customer is a good/bad on the basis of age and salary

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa

import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("03_Social_Network_Ads.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [5]:
data.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [6]:
data.isna().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [7]:
 df = data.iloc[:,1:]

In [8]:
df.duplicated().sum()

20

In [9]:
data.Purchased.value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [10]:
# Rules of Classification for ML:
# 1. Data must be complete
# 2. Data must be strictly numeric
# 3. Features and Label must be in the form of NumPy array
# 4. Features must be a 2d NP array
# 5. Label must be 1d NP array
# 6. Normalization of features is optional

In [11]:
features = data.iloc[:,[2,3]].values
label = data.iloc[:,4].values

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    label,
                                                    test_size=0.2, 
                                                    random_state=10)

In [13]:
# modeling
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

LogisticRegression()

In [14]:
# Check the quality of model
# 1. CHeck whether the model is a generalized or not
print(log_model.score(X_train,y_train))
print(log_model.score(X_test,y_test))

# 2. CHeck the quality of model with respect to CL
# SL = 0.1
# CL = 0.9

0.640625
0.65


In [15]:
# Rules for Deep Learning
# 1. Data must be complete
# 2. Data must be strictly numeric
# 3. Features and Label must be in the form of NumPy array
# 4. Features must be a 2d NP array
# 5. Label must be 2d NP array
# 6. Features must be NORMALIZED
# 7. For Binary Classification, label must be represented as 0 / 1 (pd.replace)
# 8. For Multi-class classification, label must be  DISCRETE NUMERICAL

In [16]:
features = data.iloc[:,[2,3]].values
label = data.iloc[:,[4]].values

In [17]:
from sklearn.preprocessing import RobustScaler
rbFeatures=RobustScaler()
features = rbFeatures.fit_transform(features)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    label, 
                                                    test_size=0.2,
                                                    random_state=10)

In [19]:
# Modeling
# Step1 : Architect the model
# step 1.1: Create a Sequential Model

seq_model = tf.keras.Sequential()

# Step 1.2: Create Dense Layer
# units = No. of neurons in Hidden Layer
# activation = Which Activation Function to Apply
# input_shape = Number of colums in FeatureArray


seq_model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_shape = (2,)))
seq_model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
seq_model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
seq_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [20]:
# Compiling
# Step2: Compile the mode:

seq_model.compile(optimizer="sgd", loss='binary_crossentropy', metrics=["accuracy", tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

In [21]:
#seq_model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test))

In [22]:
class MyThresholdCallBack(tf.keras.callbacks.Callback):
    def __init__(self,cl):
        super(MyThresholdCallBack, self).__init__()
        self.cl = cl
    
    def on_epoch_end(self, epoch, logs=None):
        test_score = logs["val_accuracy"]
        train_score = logs["accuracy"]
        
        if test_score > train_score and test_score > self.cl:
            self.model.stop_training = True

In [23]:
accuracy_Threshold = MyThresholdCallBack(cl=0.92)

seq_model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test), callbacks=[accuracy_Threshold])

Epoch 1/1000
10/10 [==============================] - 2s 53ms/step - loss: 0.6991 - accuracy: 0.3688 - f1_score: 0.3841 - val_loss: 0.6961 - val_accuracy: 0.4000 - val_f1_score: 0.2258
Epoch 2/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.6962 - accuracy: 0.3969 - f1_score: 0.1106 - val_loss: 0.6934 - val_accuracy: 0.4750 - val_f1_score: 0.0455
Epoch 3/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.6936 - accuracy: 0.4625 - f1_score: 0.0227 - val_loss: 0.6908 - val_accuracy: 0.6000 - val_f1_score: 0.0000e+00
Epoch 4/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.6911 - accuracy: 0.5469 - f1_score: 0.0000e+00 - val_loss: 0.6883 - val_accuracy: 0.6125 - val_f1_score: 0.0000e+00
Epoch 5/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.6888 - accuracy: 0.6281 - f1_score: 0.0000e+00 - val_loss: 0.6859 - val_accuracy: 0.6500 - val_f1_score: 0.0000e+00
Epoch 6/1000
10/10 [==============================] - 0s 8m

10/10 [==============================] - 0s 9ms/step - loss: 0.5784 - accuracy: 0.6406 - f1_score: 0.0000e+00 - val_loss: 0.5766 - val_accuracy: 0.6500 - val_f1_score: 0.0000e+00
Epoch 44/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.5756 - accuracy: 0.6406 - f1_score: 0.0000e+00 - val_loss: 0.5740 - val_accuracy: 0.6500 - val_f1_score: 0.0000e+00
Epoch 45/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.5729 - accuracy: 0.6406 - f1_score: 0.0000e+00 - val_loss: 0.5713 - val_accuracy: 0.6500 - val_f1_score: 0.0000e+00
Epoch 46/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.5702 - accuracy: 0.6406 - f1_score: 0.0000e+00 - val_loss: 0.5685 - val_accuracy: 0.6500 - val_f1_score: 0.0000e+00
Epoch 47/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.5673 - accuracy: 0.6406 - f1_score: 0.0000e+00 - val_loss: 0.5657 - val_accuracy: 0.6500 - val_f1_score: 0.0000e+00
Epoch 48/1000
10/10 [===========================

10/10 [==============================] - 0s 12ms/step - loss: 0.4271 - accuracy: 0.8188 - f1_score: 0.6882 - val_loss: 0.4128 - val_accuracy: 0.8500 - val_f1_score: 0.7391
Epoch 87/1000
10/10 [==============================] - 0s 18ms/step - loss: 0.4234 - accuracy: 0.8313 - f1_score: 0.7158 - val_loss: 0.4083 - val_accuracy: 0.8500 - val_f1_score: 0.7391
Epoch 88/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.4197 - accuracy: 0.8344 - f1_score: 0.7254 - val_loss: 0.4039 - val_accuracy: 0.8625 - val_f1_score: 0.7755
Epoch 89/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.4158 - accuracy: 0.8406 - f1_score: 0.7385 - val_loss: 0.3996 - val_accuracy: 0.8500 - val_f1_score: 0.7600
Epoch 90/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.4122 - accuracy: 0.8406 - f1_score: 0.7437 - val_loss: 0.3953 - val_accuracy: 0.8750 - val_f1_score: 0.8077
Epoch 91/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.4087 - acc

In [24]:
seq_model.evaluate(X_train,y_train)

10/10 [==============================] - 0s 3ms/step - loss: 0.3282 - accuracy: 0.8875 - f1_score: 0.8435


[0.32816630601882935, 0.887499988079071, array([0.84347826], dtype=float32)]

In [25]:
seq_model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 4ms/step - loss: 0.2917 - accuracy: 0.9250 - f1_score: 0.8966


[0.2917273938655853, 0.925000011920929, array([0.8965517], dtype=float32)]

In [26]:
(seq_model.predict(features) > 0.5).astype('int32')

13/13 [==============================] - 0s 2ms/step


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(label, (seq_model.predict(features) > 0.5).astype('int32'))

13/13 [==============================] - 0s 3ms/step


array([[235,  22],
       [ 20, 123]], dtype=int64)

In [28]:
from sklearn.metrics import classification_report
print(classification_report(label, (seq_model.predict(features) > 0.5).astype('int32')))

13/13 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.92      0.91      0.92       257
           1       0.85      0.86      0.85       143

    accuracy                           0.90       400
   macro avg       0.88      0.89      0.89       400
weighted avg       0.90      0.90      0.90       400



In [31]:
# Deployment with Input Example:

# 1. Getting user input in the form of float
ageInYears = float(input("Enter age of Customer: "))
salaryCust = float(input("Enter salary of Customer: "))

# 2. Converting input to 2d array since the same format was used for training this model
custArray = np.array([[ageInYears, salaryCust]])

# 3. Apply Standardization scince the same was done during the training of model
scaledCustData = rbFeatures.transform(custArray)

# 4. Predicting the salary
#purchase = seq_model.predict(scaledCustData)

# 5. Inverse Transform Label
actualPurch = ((seq_model.predict(scaledCustData) > 0.5).astype('int32')).item()

def final_rep(x):
    if actualPurch == 0:
        return "No"
    if actualPurch == 1:
        return "Yes"

print(f"The Customer wih {ageInYears} and salary {salaryCust} is going to purchare?: {final_rep(actualPurch)}")

Enter age of Customer: 42
Enter salary of Customer: 200000
1/1 [==============================] - 0s 48ms/step
The Customer wih 42.0 and salary 200000.0 is going to purchare?: Yes


In [30]:
scaledCustData

array([[ 0.30769231, -1.11111111]])